## Imports

In [1]:

import numpy as np
import math
from fractions import Fraction


## Constants

In [2]:
#CONSTANT

pyt_comma = 0.0136432
peaks = [2.4, 8.7, 11.4, 18.8, 29, 33.6]

#planet ratios
Schumann_earth = [7.83, 14.3, 20.8, 27.3, 33.8]
Mercury = 141.27

## Functions

### Utilities

In [3]:
def rebound(x, low = 1, high = 2, octave = 2):
    while x > high:
        x = x/octave
    while x < low:
        x = x*octave
    return x

### Peak ratios and Harmonics

In [4]:
#This function calculates the 15 ratios (with the possibility to bound them between 1 and 2) derived from the 6 peaks
def compute_peak_ratios(peaks, rebound = 1, octave = 2):
    ratios = []
    peak_ratios_rebound = []
    for p1 in peaks:
        for p2 in peaks:
            ratio_temp = p2/p1
            if ratio_temp == 1:
                ratio_temp = None
            elif ratio_temp < 1:
                ratio_temp = None

            ratios.append(ratio_temp)

        peak_ratios = np.array(ratios)
        peak_ratios = [i for i in peak_ratios if i]
        peak_ratios = list(set(peak_ratios))
    if rebound == 1:
        for peak in peak_ratios:
            while peak > octave:
                peak = peak/octave
            peak_ratios_rebound.append(peak)
    peak_ratios_rebound = np.array(peak_ratios_rebound)
    peak_ratios_rebound = [i for i in peak_ratios_rebound if i]
    peak_ratios = sorted(peak_ratios)
    peak_ratios_rebound = sorted(peak_ratios_rebound)
    return peak_ratios, peak_ratios_rebound

In [13]:
#This function takes a list of frequency peaks as input and computes the desired number of harmonics
#with the formula: x + 2x + 3x ... + nx
def EEG_harmonics_mult(peaks, n_harmonics, n_oct_up = 0, rebound = 1):
    n_harmonics = n_harmonics + 2
    multi_harmonics = []
    multi_harmonics_rebound = []
    for p in peaks:
        multi_harmonics_r = []
        multi_harm_temp = []
        harmonics = []
        p = p * (2**n_oct_up)
        i = 1
        harm_temp = p
        while i < n_harmonics:
            harm_temp = p * i
            harmonics.append(harm_temp)
            i+=1
        multi_harmonics.append(harmonics)
    multi_harmonics = np.array(multi_harmonics)  
        
    return multi_harmonics

#This function takes a list of frequency peaks as input and computes the desired number of harmonics
#with the formula: x + x/2 + x/3 ... + x/n
def EEG_harmonics_div(peaks, n_harmonics, n_oct_up = 0):
    n_harmonics = n_harmonics + 2
    multi_harmonics = []
    multi_harmonics_sub = []
    for p in peaks:
                           
        harmonics = [] 
        harmonics_sub = []
        p = p * (2**n_oct_up)               
        i = 2
        harm_temp = p
        harm_temp_sub = p
        while i < n_harmonics:
            harm_temp = harm_temp + (p/i)
            harm_temp_sub = abs(harm_temp_sub - (p/i))
            harmonics.append(harm_temp)
            harmonics_sub.append(harm_temp_sub)
            i+=1    
        multi_harmonics.append(harmonics)
        multi_harmonics_sub.append(harmonics_sub)
    multi_harmonics = np.array(multi_harmonics)
    multi_harmonics_bounded = multi_harmonics.copy()
    multi_harmonics_sub = np.array(multi_harmonics_sub)
    multi_harmonics_sub_bounded = multi_harmonics_sub.copy()
    #Rebound the result between 1 and 2
    for i in range(len(multi_harmonics_bounded)):
        for j in range(len(multi_harmonics_bounded[0])):
            multi_harmonics_bounded[i][j] = rebound(multi_harmonics_bounded[i][j])
            multi_harmonics_sub_bounded[i][j] = rebound(multi_harmonics_sub_bounded[i][j])
    return multi_harmonics, multi_harmonics_bounded, multi_harmonics_sub, multi_harmonics_sub_bounded

In [15]:
mh, mh_bound, mh_sub, mh_sub_bound = EEG_harmonics_div(peaks, 5, n_oct_up = 0)
mh_sub_bound, mh_bound

(array([[1.2       , 1.6       , 1.6       , 1.12      , 1.92      ],
        [1.0875    , 1.45      , 1.45      , 1.015     , 1.74      ],
        [1.425     , 1.9       , 1.9       , 1.33      , 1.14      ],
        [1.175     , 1.56666667, 1.56666667, 1.09666667, 1.88      ],
        [1.8125    , 1.20833333, 1.20833333, 1.69166667, 1.45      ],
        [1.05      , 1.4       , 1.4       , 1.96      , 1.68      ]]),
 array([[1.8       , 1.1       , 1.25      , 1.37      , 1.47      ],
        [1.63125   , 1.99375   , 1.1328125 , 1.2415625 , 1.3321875 ],
        [1.06875   , 1.30625   , 1.484375  , 1.626875  , 1.745625  ],
        [1.7625    , 1.07708333, 1.22395833, 1.34145833, 1.439375  ],
        [1.359375  , 1.66145833, 1.88802083, 1.03463542, 1.11015625],
        [1.575     , 1.925     , 1.09375   , 1.19875   , 1.28625   ]]))

### n-TET 

In [ ]:


def oct_subdiv(ratio,bounds,octave,n):    
    Octdiv, Octvalue, i = [], [], 1
    ratios = []
    while len(Octdiv) < n:
        ratio_mult = (ratio**i)
        while ratio_mult > octave:
            ratio_mult = ratio_mult/octave    
            
        rescale_ratio = ratio_mult - round(ratio_mult)
        ratios.append(ratio_mult)
        i+=1
        if -bounds < rescale_ratio < bounds:
            Octdiv.append(i-1)
            Octvalue.append(ratio_mult)
        else:
            continue
    return Octdiv, Octvalue, ratios


def compare_octave_division(Octdiv = 53, Octdiv2 = 12, bounds = 0.01, octave = 2):
    ListOctdiv = []
    ListOctdiv2 = []
    OctdivSum = 1
    OctdivSum2 = 1
    i = 1
    i2 = 1
    Matching_harmonics = []
    #HARMONIC_RATIOS = [1, 1.0595, 1.1225, 1.1892, 1.2599, 1.3348, 1.4142, 1.4983, 1.5874, 1.6818, 1.7818, 1.8897]
    while OctdivSum < octave:
        OctdivSum =(nth_root(octave, Octdiv))**i
        i+=1
        ListOctdiv.append(OctdivSum)
    #print(ListOctdiv)

    while OctdivSum2 < octave:
        OctdivSum2 =(nth_root(octave, Octdiv2))**i2
        i2+=1
        ListOctdiv2.append(OctdivSum2)
    #print(ListOctdiv2)
    for i, n in enumerate(ListOctdiv):
        for j, harm in enumerate(ListOctdiv2):
            if harm-bounds < n < harm+bounds:
                Matching_harmonics.append([n, i+1, harm, j+1])
    Matching_harmonics = np.array(Matching_harmonics)
    return Matching_harmonics

def nth_root (num, root):
    answer = num**(1/root)
    return answer


### Consonance

In [22]:
#Oct_subdiv
#Argument 1 : a ratio in the form of a float or a fraction
#Argument 2 : bounds between which the octave should fall
#Argument 3 : value of the octave
#Argument 4 : number of octave subdivisions
def compute_consonance (peaks, limit):
    consonance_ = []
    peaks2keep = []
    peaks_consonance = []
    for p1 in peaks:    
        for p2 in peaks:
            peaks2keep_temp = []
            p2x = p2
            p1x = p1
            cons_temp = (p2/p1).as_integer_ratio()
            cons_temp = (cons_temp[0] + cons_temp[1])/(cons_temp[0] * cons_temp[1])
            #print(cons_temp)
            if cons_temp == 2:
                cons_temp = None
                p2x = None
                p1x = None
            elif cons_temp < limit:
                cons_temp = None
                p2x = None
                p1x = None
            if p2x != None:
                peaks2keep_temp.extend([p2x, p1x])
            consonance_.append(cons_temp)  
            peaks2keep.append(peaks2keep_temp)
        peaks_consonance = np.array(peaks2keep)
        peaks_consonance = [x for x in peaks_consonance if x]
        consonance = np.array(consonance_)
        consonance = [i for i in consonance if i]
        #consonance = list(set(consonance))
    return consonance, peaks_consonance

# Function that computes integer ratios from peaks with higher consonance
def comp_consonant_integers_ratios (peaks_consonance):
    cons_integer = []
    for i in range(len(peaks_consonance)):
        #for j in range(len(peaks_consonance[0])):
        a = peaks_consonance[i][0]
        b = peaks_consonance[i][1]
        if a > b:
            while a > (b*2):
                a = a/2
        elif a < b:
            while b > (a*2):
                b = b/2
        cons_temp = (a/b).as_integer_ratio()
        cons_integer.append(cons_temp)
    consonant_integers = np.array(cons_integer).squeeze()
    cons_ratios = []
    for j in range(len(consonant_integers)):
        cons_ratios.append((consonant_integers[j][0])/(consonant_integers[j][1]))
    consonant_ratios = np.array(cons_ratios)
    return consonant_integers, consonant_ratios

In [ ]:
a, b = compute_consonance(peaks + Schumann_earth, 0.05)
c, d = comp_consonant_integers_ratios (b)
d

### Tunings

In [17]:
# Function that computes Pythogorian Tunings from integer ratios 
# List of equations of the Pythagorean Tuning
# variable 'a' has to be the highest value in the ratio

#INPUTS (a: numerator of the source ratio / b: denominator of the source ratio
#        n: multiplifier of all notes / octa: octave value / lim_denom: highest value possible
#        for the denominator of the ratios)
#OUTPUTS

def compute_pythagore(a = 3, b = 2, n = 1, octa = 2, lim_denom = 1000):
    import operator
    pyth0 = n * (1/1)
    pyth1 = n * ((b/a)**5)*(octa**3)
    pyth1 = rebound(pyth1)
    pyth2 = n * ((a/b)**2)*(1/octa)
    pyth2 = rebound(pyth2)
    pyth3 = n * ((b/a)**3)*(octa**2)
    pyth3 = rebound(pyth3)
    pyth4 = n * ((a/b)**4)*((1/octa)**2)
    pyth4 = rebound(pyth4)
    pyth5 = n * (b/a)*octa
    pyth5 = rebound(pyth5)
    pyth6 = n * ((b/a)**6)*(octa**4)
    pyth6 = rebound(pyth6)
    pyth6_2 = n * ((a/b)**6)*((1/octa)**3)
    pyth6_2 = rebound(pyth6_2)
    pyth7 = n * (a/b)
    pyth7 = rebound(pyth7)
    pyth8 = n * ((b/a)**4)*(octa**3)
    pyth8 = rebound(pyth8)
    pyth9 = n * ((a/b)**3)*(1/octa)
    pyth9 = rebound(pyth9)
    pyth10 = n * ((b/a)**2)*(octa**2)
    pyth10 = rebound(pyth10)
    pyth11 = n * ((a/b)**5)*((1/octa)**2)
    pyth11 = rebound(pyth11)
    
    pyth_dict = {'unison':pyth0,
             'minor second':pyth1,
             'major second':pyth2,
             'minor third':pyth3,
             'major third':pyth4,
             'perfect fourth':pyth5,
             'diminished fifth':pyth6,
             'augmented fourth':pyth6_2,
             'perfect fifth':pyth7,
             'minor sixth':pyth8,
             'major sixth':pyth9,
             'minor seventh':pyth10,
             'major seventh':pyth11}
    pyth_dict_list = sorted(pyth_dict.items(), key=operator.itemgetter(1))
    pyth_dict = dict(pyth_dict)
    pyth_temp = []
    pyth_temp_float = []
    for i in range(len(pyth_dict_list)):
        frac = []
        #print(harm_tuning[i])
        pyth_ratios = Fraction(pyth_dict_list[i][1]).limit_denominator(lim_denom)
        pyth_float = pyth_dict_list[i][1]
        num = pyth_ratios.numerator
        denom = pyth_ratios.denominator
        frac.extend([num, denom])
        #print(harm_ratios_temp)
        pyth_temp.append(frac)
        pyth_temp_float.append(pyth_float)
       
    pyth_dict_list_ratios = (np.array(pyth_temp)).squeeze()
    pyth_dict_list_float = np.array(pyth_temp_float)
    
    return pyth_dict, pyth_dict_list, pyth_dict_list_ratios, pyth_dict_list_float

#This Function build a tuning based on the multiplication of a ratio by itself.
def harmonic_tuning(pos = 1, n_harmonics = 10, octave = 2):
    harm_tuning_temp = []
    i = 2
    harm_ratios = []
    harm_ratios_temp = []
    while i < n_harmonics:
        harm_temp = pos*i
        harm_tuning_temp.append(harm_temp)
        i+=1
    
    for h in harm_tuning_temp:
                while h > pos*octave:
                    h = h/octave
                while h > octave:
                    h = h/pos
                h = round(h, 6)

                if h == octave:
                    h = None
                harm_ratios_temp.append(h)
    harm_tuning = np.array(harm_ratios_temp)
    harm_tuning = [i for i in harm_tuning if i]
    harm_tuning = list(set(harm_tuning))
    for i in range(len(harm_tuning)):
        frac = []
        #print(harm_tuning[i])
        harm_ratios_temp = Fraction(harm_tuning[i]).limit_denominator()
        num = harm_ratios_temp.numerator
        denom = harm_ratios_temp.denominator
        frac.extend([num, denom])
        #print(harm_ratios_temp)
        harm_ratios.append(frac)
    
    harm_tuning_frac = (np.array(harm_ratios)).squeeze()

    return harm_tuning, harm_tuning_frac

## Bidouille

In [23]:
#compute_consonance takes a list of peaks as input(1) and a threshold(2)
#above which the peaks are kept
#output(1) returns the consonance values
#output(2) return the peaks corresponding to the consonance values
consonant_values, consonant_peaks = compute_consonance(Schumann_earth+peaks, 0.01)
print('CONSONANT PEAKS')
print(consonant_peaks)
#This function takes a list of consonant peaks and returns the peak ratios
#with integer values
cons_integers = comp_consonant_integers_ratios(consonant_peaks)
print('CONSONANT INTEGER RATIOS')
print(cons_integers)

CONSONANT PEAKS
[[14.3, 20.8], [27.3, 20.8], [27.3, 2.4], [8.7, 2.4], [11.4, 2.4], [27.3, 33.6]]
CONSONANT INTEGER RATIOS
(array([[11, 16],
       [21, 16],
       [91, 64],
       [29, 16],
       [19, 16],
       [13, 16]]), array([0.6875  , 1.3125  , 1.421875, 1.8125  , 1.1875  , 0.8125  ]))


C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In [ ]:

#Oct_subdiv
#Argument 1 : a ratio in the form of a float or a fraction
#Argument 2 : bounds between which the octave should fall
#Argument 3 : value of the octave
#Argument 4 : number of octave subdivisions
Octdiv, Octvalue, ratios_tet = oct_subdiv(1.5, 0.02, 2, 1)
print('OCTAVE SUBDIVISIONS')
print(Octdiv, Octvalue)
print('RATIOS-TET')
print(ratios_tet)
pairwise_oct_div = compare_octave_division(Octdiv = 4, Octdiv2 = 14, bounds = 0.01, octave = 2)
print('COMPARE OCTAVE SUBDIVISIONS')
print(pairwise_oct_div)




peak_ratios, peak_ratios_rebound = compute_peak_ratios(peaks, 1)
print('PEAK RATIOS')
print(peak_ratios, peak_ratios_rebound)



#Calculates a specific number of partials from a list of peaks [n_peaks][n_harmonic]
multi_harmonics= EEG_harmonics(peaks, 12, 0)
print('HARMONICS')
print(multi_harmonics)
#Calculates a specific number of elements in the harmonics series
#from a list of peaks [n_peaks][n_harmonic]
harm_div, harm_div_bound, harm_div_sub, harm_div_sub_bound = EEG_harmonics_div(peaks, n_harmonics = 8)
print('HARMONIC SERIES')
print(harm_series_rebound)

#pyth_dict, pyth_dict_list = compute_pythagore(cons_integers[2][0], cons_integers[2][1], 1, 2)
a, b, c, d = compute_pythagore(3, 2, 1, 2, 1000)
a

In [ ]:
a, b, c, d = compute_pythagore(4, 3, 1, 2, 1000)
b

In [ ]:
#Create a function that compute pair-wise comparisons between each harmonic of one peak with all harmonics of other peaks
#and returns matching peaks and their harmonic position
bounds = 0.5
for p in range(len(multi_harmonics)):
    for h in range(len(multi_harmonics[0])):
        if p != 
        
        

In [ ]:
a, b, c, d= compute_pythagore(7, 4, 1, 2)
d

In [ ]:
1.25-1.265625
81/80
(81/64)-(5/4)

In [18]:
h, i = harmonic_tuning(10, 40, 2)
h, i


([1.5,
  1.125,
  1.375,
  1.25,
  1.75,
  1.625,
  1.875,
  1.0625,
  1.1875,
  1.3125,
  1.4375,
  1.5625,
  1.6875,
  1.8125,
  1.9375,
  1.15625,
  1.21875,
  1.09375,
  1.03125],
 array([[ 3,  2],
        [ 9,  8],
        [11,  8],
        [ 5,  4],
        [ 7,  4],
        [13,  8],
        [15,  8],
        [17, 16],
        [19, 16],
        [21, 16],
        [23, 16],
        [25, 16],
        [27, 16],
        [29, 16],
        [31, 16],
        [37, 32],
        [39, 32],
        [35, 32],
        [33, 32]]))

In [ ]:
multi_harmonics = EEG_harmonics(peaks, 50, 0)
multi_harmonics

In [ ]:


a, b, ratio_mult = oct_subdiv(1.5, 0.02, 2, 1)
print(ratio_mult)


pyth_dict, pyth_dict_list = compute_pythagore(3, 2, 1, 2)
print(pyth_dict_list)

In [ ]:
## Calculate hertz to cents

c = 1200 × log2 (f2 / f1)

# PyTuning

In [ ]:
import pytuning as pt
import sympy as sp
from pytuning.scales import create_edo_scale
edo_12_scale = create_edo_scale(53)
edo_12_scale

x = pt.find_best_modes(peak_ratios_rebound, 7)
x = pt.find_best_modes(ratios_tet, 7)
x
#major_mask = (0,2,4,5,7,9,11,12)
#major_mode = mask_scale(edo_12_scale, major_mask)
#major_mode

In [ ]:
peak_ratios_rebound

## PyAudio

In [ ]:
def notes2play(array, fund):
    scale2play = []
    scale2play = [fund*x for x in array]
    
    return scale2play
    

In [ ]:
import pyaudio
import struct
import math

FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100

p = pyaudio.PyAudio()


def data_for_freq(frequency: float, time: float = None):
    """get frames for a fixed frequency for a specified time or
    number of frames, if frame_count is specified, the specified
    time is ignored"""
    frame_count = int(RATE * time)

    remainder_frames = frame_count % RATE
    wavedata = []

    for i in range(frame_count):
        a = RATE / frequency  # number of frames per wave
        b = i / a
        # explanation for b
        # considering one wave, what part of the wave should this be
        # if we graph the sine wave in a
        # displacement vs i graph for the particle
        # where 0 is the beginning of the sine wave and
        # 1 the end of the sine wave
        # which part is "i" is denoted by b
        # for clarity you might use
        # though this is redundant since math.sin is a looping function
        # b = b - int(b)

        c = b * (2 * math.pi)
        # explanation for c
        # now we map b to between 0 and 2*math.PI
        # since 0 - 2*PI, 2*PI - 4*PI, ...
        # are the repeating domains of the sin wave (so the decimal values will
        # also be mapped accordingly,
        # and the integral values will be multiplied
        # by 2*PI and since sin(n*2*PI) is zero where n is an integer)
        d = math.sin(c) * 32767
        e = int(d)
        wavedata.append(e)

    for i in range(remainder_frames):
        wavedata.append(0)

    number_of_bytes = str(len(wavedata))  
    wavedata = struct.pack(number_of_bytes + 'h', *wavedata)

    return wavedata


def play(frequency: float, time: float):
    """
    play a frequency for a fixed time!
    """
    frames = data_for_freq(frequency, time)
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True)
    stream.write(frames)
    stream.stop_stream()
    stream.close()


#if __name__ == "__main__":
    
    
        


In [ ]:
scale2play = notes2play(d, 100)
scale2play
for i in scale2play:
    play(i, 1)
    